# Physics-informed neural network (PINN) example with harmonic oscillator

In this demo we will code a PINN from scratch in `PyTorch` and use it to solve simulation and inversion problems related to the damped harmonic oscillator.

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

## Function Approximation with PyTorch

The objective of this tutorial is learn the basics of PyTorch.

First, we cosider the task of learning a function, with two simple examples:

Polynomial class (considered before in class):
$$f(x) =2x-10x^5+15x^{10}, \quad x\in(0, 1)$$

Damped oscillations (example for PINN):
$$ f(x) = e^{-x/\delta} \sin(\omega x), \quad x\in(0, 2 \pi) $$

by using feed-forward dense neural networks.

Three Steps:
1.  Dataset generation
2.  Build the Pytorch Model
3.  Pytorch training loop

### $\mathbf{\text{Step 1: Dataset Generation}}$

Consider a probabilistic process that gives rise to labeled data $(x,y)$. The training data is generated by drawing samples from the equation
$$
    y_i= f(x_i) + \eta_i,
$$
where $f(x_i)$ is some fixed function, and $\eta_i$ is a Gaussian, uncorrelated noise variable such that
$$
\langle \eta_i \rangle=0 
$$
$$
\langle \eta_i \eta_j \rangle = \delta_{ij} \sigma
$$
We will refer to the $f(x_i)$ as the **true model** used to generate the data. 


In [16]:
#generate two kinds of training data, we will work with the polynomial data first
def generate_data_polynomial(num_points, noise_std):
    # Generate random x values
    max_x = 1
    x_values = np.sort(max_x*np.random.random(num_points))
    # Generate corresponding y values for polynomial with added noise
    y_values =2*x_values-10*x_values**5+15*x_values**10 + np.random.normal(0, noise_std, num_points)
    return x_values, y_values

def generate_data_damped_oscillation(num_points, noise_std):   
    x_values = np.sort(np.random.uniform(0, 2 * np.pi, num_points))
    # generate damped oscillation 
    y_values = np.sin(4*x_values)*np.exp(-x_values/2.0) + np.random.normal(0, noise_std, num_points)
    return x_values, y_values

In [ ]:
num_points = 100
# You can adjust the noise level as needed
x_train, y_train = generate_data_polynomial(num_points, noise_std=0.1)
# Plot the generated data for polynomial
plt.figure(figsize=(10, 6))
plt.scatter(
    x_train, y_train,
    label='Generated Data with Noise', color='blue', alpha=0.5)
plt.plot(
    x_train, 2*x_train-10*x_train**5+15*x_train**10,
    label='True Polynomial', color='red', linestyle='--')

plt.legend()
plt.title('Generated Polynomial Data with Noise')
plt.xlabel('X values')
plt.ylabel('Y values')
plt.show()

In [1]:
# Generate and plot data for damped oscillations


$\mathbf{\text{Step 2: Model Definition}}$

We use feedforward neural (also named as a multi-layer perceptron) network to approximate the function  $f(x)$
Given an input $x \in D \subset R^n$, a feedforward neural network transforms it to an output $NN_\theta(x)\in R^m$, through a layer of units (neurons) which compose of either affine-linear maps between units (in successive layers) or scalar non-linear activation functions within units, resulting in the representation,

$$NN_{\theta}(x) = C_K \circ A \circ C_{K-1}\ldots \ldots \ldots \circ A\circ C_2 \circ A\circ C_1(x).$$

Here, $\circ$ refers to the composition of functions and $A$ is a scalar (non-linear) activation function. For any $1 \leq k \leq K$, we define

$$
C_k z_k = W_k z_k + b_k, \quad {\rm for} ~ W_k \in R^{d_{k+1} \times d_k}, z_k \in R^{d_k}, b_k \in R^{d_{k+1}}.
$$

We also denote,
$$
\theta = \{W_k, b_k\}, \theta_W = \{ W_k \}\quad \forall~ 1 \leq k \leq K,
$$
to be the complete set of (tunable) weights for our network.


In [23]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(1, 10)  # 1 input feature, 10 hidden units
        self.activation = nn.SiLU()
        self.fc2 = nn.Linear(10, 1)  # 10 hidden units, 1 output

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x

$\mathbf{\text{Step 3: Model Training}}$

The neural network $u_{\theta}$ depends on the tuning parameter $\theta \in \Theta$ of weights and biases. Within the standard paradigm of deep learning, one trains the network by finding tuning parameters $\theta$ such that a suitable loss function $J(\theta)$ is minimized.

$${\rm Find}~\theta^{\ast} \in \Theta:\quad \theta^{\ast} = {\rm arg}\min\limits_{\theta \in \Theta} \left( J(\theta)\right).$$

The loss function, for instance, can be chosen as the mean square error between the neural network and the underlying target:

$$ J(\theta) = \sum_{i}^n \Big(u_i - u_\theta(x_i)\Big)^2$$

The optimization process is realized with the gradient descent (or more precisely with variants of the gradient descent such as Adam or SGD).

1. Compute the loss function over the batch $j$:
$J_S(\theta)=\sum_{x_i \in {S}_j}^n \Big(u_i - u_\theta(x_i)\Big)^2$

2. Compute the gradient of $J_S(\theta)$ with respect to the network parameters:  $\nabla_\theta J_S(\theta)$

3. Update the parameters according to the chosen optimizer, for instance for minibatch stochastic gradient descent $\theta_{k+1} = \theta_{k} - \eta \nabla_\theta J_S(\theta_{k}) $ with $k=1,...,(n_{epoch} n_{batch})$ and $\eta$ being the learning rate (argument $lr$ in the optimizer).

In [24]:
def get_optimizer(optimizer_name, model):
  if optimizer_name == "ADAM":
      return optim.Adam(model.parameters())
  elif optimizer_name == "LBFGS":
      return optim.LBFGS(model.parameters())
  raise ValueError('Chosen optimizer is unavailable.')


def train(model, optimizer, criterion, inputs, targets):
    def closure():
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        return loss

    optimizer.step(closure)
    return closure().item()


def evaluate(model, x_test):
    model.eval()
    with torch.no_grad():
        y_pred = model(x_test.view(-1, 1)).numpy()
    return y_pred

#### Instantiate your model, define your loss function + optimizer, and perform the training procedure.

In [ ]:
model = SimpleNN()
criterion = nn.MSELoss()
optimizer = get_optimizer('ADAM', model)

num_epochs = 10000
for epoch in range(num_epochs):
    # key step! Convert data to PyTorch tensors, a special data structure for Pytorch (similar to ndarrays for numpy)
    inputs = torch.tensor(
        x_train, dtype=torch.float32, requires_grad=True).view(-1, 1)
    targets = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

    # Training
    loss = train(model, optimizer, criterion, inputs, targets)

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')

$\mathbf{\text{Step 4: Plotting Predictions}}$
Final step, evaluate your predictions at some input locations and do some plotting.

In [ ]:
x_test = torch.linspace(0, 1., 100)
y_pred = evaluate(model, x_test)

plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, label='Original Noisy Training Data', color='orange', alpha=1)
plt.plot(x_test,2*x_test-10*x_test**5+15*x_test**10, label='Original True Polynomial', color='blue', alpha=1.0)
plt.plot(x_test, y_pred, label='Learned Curve', color='red', alpha=1.0)

plt.legend()
plt.show()

Interesting parameters to play with:  Size of training data and noise
number of learning steps (epochs) and learning rate

### Your turn:
Compare training, testing and predicted data for damped oscillator

In [ ]:
x_test = torch.linspace(0, 1.2*2*np.pi, 100)
y_pred = evaluate(model, x_test)

plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, label='Original Noisy Training Data', color='orange', alpha=1)
plt.plot(x_test,np.sin(4*x_test)*np.exp(-x_test/2.0), label='Original Damped Oscillation', color='blue')
plt.plot(x_test, y_pred, label='Learned Curve', color='red')

plt.legend()
plt.show()

## Physics-Informed Neural Network (PINN) Problem overview

This example builds upon the blog post tutorial by Ben Moseley on PINNs: https://benmoseley.blog/my-research/so-what-is-a-physics-informed-neural-network/. 

Read the seminal PINN papers [here](https://ieeexplore.ieee.org/document/712178) and [here](https://www.sciencedirect.com/science/article/pii/S0021999118307125).

In this demo we will code a PINN from scratch in  **PyTorch** and use it to solve simulation and inversion problems related to the damped harmonic oscillator.

We are going to use a PINN to solve problems related to the **damped harmonic oscillator**:

<img src="oscillator.gif" width="500">

We are interested in modelling the displacement of the mass on a spring (green box) over time.

This is a canonical physics problem, where the displacement, $u(t)$, of the oscillator as a function of time can be described by the following differential equation:

$$
m \dfrac{d^2 u}{d t^2} + \mu \dfrac{d u}{d t} + ku = 0~,
$$

where $m$ is the mass of the oscillator, $\mu$ is the coefficient of friction and $k$ is the spring constant.

We will focus on solving the problem in the **under-damped state**, i.e. where the oscillation is slowly damped by friction (as displayed in the animation above). 

Mathematically, this occurs when:

$$
\delta < \omega_0~,~~~~~\mathrm{where}~~\delta = \dfrac{\mu}{2m}~,~\omega_0 = \sqrt{\dfrac{k}{m}}~.
$$

Furthermore, we consider the following initial conditions of the system:

$$
u(t=0) = 1~~,~~\dfrac{d u}{d t}(t=0) = 0~.
$$

For this particular case, the exact solution is known and given by:

$$
u(t) = e^{-\delta t}(2 A \cos(\phi + \omega t))~,~~~~~\mathrm{with}~~\omega=\sqrt{\omega_0^2 - \delta^2}~.
$$



For a more detailed mathematical description of the harmonic oscillator, check out this blog post: https://beltoforion.de/en/harmonic_oscillator/.

## Workflow overview

There are **two scientific tasks** related to the harmonic oscillator we will use a PINN for:

>First, we will **simulate** the system using a PINN, given its initial conditions.

>Second, we will **invert** for underlying parameters of the system using a PINN, given some noisy observations of the oscillator's displacement.


## Initial setup

First, we define a few helper functions.

The constructor initializes the model with:

N_INPUT: Number of input features

N_OUTPUT: Number of output features

N_HIDDEN: Number of hidden units per layer

N_LAYERS: Total number of layers including input and output



In [35]:
def exact_solution(d, w0, t):
    "Defines the analytical solution to the under-damped harmonic oscillator problem above."
    assert d < w0
    w = np.sqrt(w0**2-d**2)
    # this expression for the phase phi is from the initial condition: du/dt (t=0) = 0
    phi = np.arctan(-d/w)
    #this expression of A is from the initial condition: u(t=0) = 1:
    A = 1/(2*np.cos(phi))
    cos = torch.cos(phi+w*t)
    exp = torch.exp(-d*t)
    u = exp*2*A*cos
    return u

#This class FCN inherits from torch.nn.Module, the base class for all neural network models in PyTorch.
class FCN(nn.Module):
    "Defines a fully-connected network in PyTorch"
    def __init__(self, N_INPUT, N_OUTPUT, N_HIDDEN, N_LAYERS):
        super().__init__()
        activation = nn.Tanh
        # self.fcs is the input layer followed by a Tanh activation.
        self.fcs = nn.Sequential(*[
                        nn.Linear(N_INPUT, N_HIDDEN),
                        activation()])
        #self.fch is a sequence of (N_LAYERS - 1) identical blocks 
        # each containing a linear layer and a tanh activation
        self.fch = nn.Sequential(*[
                        nn.Sequential(*[
                            nn.Linear(N_HIDDEN, N_HIDDEN),
                            activation()]) for _ in range(N_LAYERS-1)])
        #output layer maps hidden activation to output features
        self.fce = nn.Linear(N_HIDDEN, N_OUTPUT)
    def forward(self, x):
        x = self.fcs(x)  # Apply input layer and activation
        x = self.fch(x)  # Pass through hidden layers
        x = self.fce(x)  #Apply output layer
        return x

## Task 1: train a PINN to simulate the system

#### Task

The first task is to use a PINN to **simulate** (solve) the system.

Specifically, our inputs and outputs are:

- Inputs: underlying differential equation and the initial conditions of the system
- Outputs: estimate of the solution, $u(t)$

#### Approach

The PINN is trained to directly approximate the solution to the differential equation, i.e.

$$
N\!N(t;\theta) \approx u(t)~,
$$

For this task, we use $\delta=2$, $\omega_0=20$, $m=1$, and try to learn the solution over the domain $t\in [0,1]$.

#### Loss function

To simulate the system, the PINN is trained with the following loss function:

$$
\mathcal{L}(\theta)= (N\!N(0;\theta) - 1)^2 + \lambda_1 \left(\frac{d N\!N}{dt}(0;\theta) - 0\right)^2 + \frac{\lambda_2}{N} \sum^{N}_{i} \left( \left[ m\frac{d^2}{dt^2} + \mu \frac{d}{dt} + k \right] N\!N(t_{i};\theta)  \right)^2
$$

where $\lambda_1$ and $\lambda_2$ are hyperparameters used for regularization of the training.

The first two terms in the loss function represent the boundary loss, and tries to ensure that the solution learned by the PINN matches the initial conditions of the system, namely.
The second term in the loss function is called the physics loss, and and tries to ensure that the PINN solution obeys the underlying differential equation at a set of training points 
sampled over the entire domain.

#### Computing gradients

To compute gradients of the neural network with respect to its inputs, we will use `torch.autograd.grad`

In [ ]:
torch.manual_seed(123)

# define a neural network to train  ( Fully-connected Network)
pinn = FCN(1,1,32,3)

# define boundary points, for the boundary loss
t_boundary = torch.tensor(0.).view(-1,1).requires_grad_(True)# (1, 1)

# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)# (30, 1)

# train the PINN
d, w0 = 2, 20
mu, k = 2*d, w0**2
t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(d, w0, t_test)
#specify optimizer choice and learning rate
optimiser = torch.optim.Adam(pinn.parameters(),lr=1e-3)
for i in range(15001):
    optimiser.zero_grad()
    
    # compute each term of the PINN loss function above
    # using the following hyperparameters
    lambda1, lambda2 = 1e-1, 1e-4
    
    # compute boundary loss
    u = pinn(t_boundary)# (1, 1)
    loss1 = (torch.squeeze(u) - 1)**2
    dudt = torch.autograd.grad(u, t_boundary, torch.ones_like(u), create_graph=True)[0]# (1, 1)
    loss2 = (torch.squeeze(dudt) - 0)**2
    
    # compute physics loss
    u = pinn(t_physics)# (30, 1)
    dudt = torch.autograd.grad(u, t_physics, torch.ones_like(u), create_graph=True)[0]# (30, 1)
    d2udt2 = torch.autograd.grad(dudt, t_physics, torch.ones_like(dudt), create_graph=True)[0]# (30, 1)
    loss3 = torch.mean((d2udt2 + mu*dudt + k*u)**2)
    
    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2 + lambda2*loss3
    loss.backward()
    optimiser.step()
    
    # plot the result as training progresses
    if i % 5000 == 0: 
        #print(u.abs().mean().item(), dudt.abs().mean().item(), d2udt2.abs().mean().item())
        u = pinn(t_test).detach()
        plt.figure(figsize=(6,2.5))
        plt.scatter(t_physics.detach()[:,0], 
                    torch.zeros_like(t_physics)[:,0], s=20, lw=0, color="tab:green", alpha=0.6)
        plt.scatter(t_boundary.detach()[:,0], 
                    torch.zeros_like(t_boundary)[:,0], s=20, lw=0, color="tab:red", alpha=0.6)
        plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        plt.show()

## Task 2: train a PINN to invert for underlying parameters

#### Task

The second task is to use a PINN to **invert** for underlying parameters.

Specifically, our inputs and outputs are:

- Inputs: noisy observations of the oscillator's displacement, $u_{\mathrm{obs}}$
- Outputs: estimate $\mu$, the coefficient of friction

#### Approach

Similar to above, the PINN is trained to directly approximate the solution to the differential equation, i.e.

$$
N\!N(t;\theta) \approx u(t)~,
$$

However here we assume $\mu$ is **not known** and we treat it as an additional **learnable parameter** when training the PINN.

#### Loss function

The PINN is trained with the loss function:

$$
\mathcal{L}(\theta, \mu)= \frac{1}{N} \sum^{N}_{i} \left( \left[ m\frac{d^2}{dt^2} + \mu \frac{d}{dt} + k \right] N\!N(t_{i};\theta)  \right)^2 + \frac{\lambda}{M} \sum^{M}_{j} \left( N\!N(t_{j};\theta) - u_{\mathrm{obs}}(t_{j}) \right)^2
$$

There are two terms in the loss function here. The first is the physics loss, formed in the same way as above, which ensures the solution learned by the PINN is consistent with the know physics.

The second term is called the data loss, and makes sure that the solution learned by the PINN fits the (potentially noisy) observations of the solution that are available.

Note, we have removed the boundary loss terms, as we do not know these (i.e., we are only given the observed measurements of the system).

In this set up, the PINN parameters $\theta$ and $\mu$ are jointly learned during optimisation.

In [ ]:
# first, create some noisy observational data
torch.manual_seed(123)
d, w0 = 2, 20
print(f"True value of mu: {2*d}")
t_obs = torch.rand(40).view(-1,1)
u_obs = exact_solution(d, w0, t_obs) + 0.04*torch.randn_like(t_obs)
t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

plt.figure(figsize=(6,2.5))
plt.title("Noisy observational data")
plt.scatter(t_obs[:,0], u_obs[:,0])
plt.plot(t_test[:,0], u_exact[:,0], label="Exact solution", color="tab:grey", alpha=0.6)
plt.show()

In [ ]:
torch.manual_seed(123)

# define a neural network to train
pinn = FCN(1,1,32,3)

# define training points over the entire domain, for the physics loss
t_physics = torch.linspace(0,1,30).view(-1,1).requires_grad_(True)# (30, 1)

# train the PINN
d, w0 = 2, 20
_, k = 2*d, w0**2
t_test = torch.linspace(0,1,300).view(-1,1)
u_exact = exact_solution(d, w0, t_test)

# treat mu as a learnable parameter, add it to optimiser
mu = torch.nn.Parameter(torch.zeros(1, requires_grad=True))
optimiser = torch.optim.Adam(list(pinn.parameters())+[mu],lr=1e-3)
mus = []
for i in range(15001):
    optimiser.zero_grad()
    
    # compute each term of the PINN loss function above
    # using the following hyperparameters
    lambda1 = 1e4
    
    # compute physics loss
    u = pinn(t_physics)# (30, 1)
    dudt = torch.autograd.grad(u, t_physics, torch.ones_like(u), create_graph=True)[0]# (30, 1)
    d2udt2 = torch.autograd.grad(dudt, t_physics, torch.ones_like(dudt), create_graph=True)[0]# (30, 1)
    loss1 = torch.mean((d2udt2 + mu*dudt + k*u)**2)
    
    # compute data loss
    u = pinn(t_obs)
    loss2 = torch.mean((u - u_obs)**2)
    
    # backpropagate joint loss, take optimiser step
    loss = loss1 + lambda1*loss2
    loss.backward()
    optimiser.step()
    
    # record mu value
    mus.append(mu.item())
    
    # plot the result as training progresses
    if i % 5000 == 0: 
        u = pinn(t_test).detach()
        plt.figure(figsize=(12,2.5))
        
        plt.subplot(1,2,1)
        plt.scatter(t_obs[:,0], u_obs[:,0], label="Noisy observations", alpha=0.6, color="tab:blue")
        plt.plot(t_test[:,0], u[:,0], label="PINN solution", color="tab:green")
        plt.title(f"Training step {i}")
        plt.legend()
        
        plt.subplot(1,2,2)
        plt.title("$\mu$")
        plt.plot(mus, label="PINN estimate", color="tab:green")
        plt.hlines(2*d, 0, len(mus), label="True value", color="tab:grey")
        plt.xlabel("Training step")
        plt.legend()
        
        plt.show()

Notice how the PINN estimate of $\mu$ finally converges to its true value!

## Suggested Extensions

PINNs have been extended and improved in many ways since they have been proposed. Some things to try are:

- Try extending them to higher dimensions (e.g. 2D and 3D simulations)
- See how far you can push the inversion task: can you discover $m$, $\mu$ and $k$ simultaneously (and therefore, discover the entire underlying equation?)